In [1]:
import requests
import numpy as np
import pandas as pd
import time
import pickle
import psycopg2 as pg
import pandas.io.sql as pd_sql
import sqlalchemy

In [7]:
# Do some queries to get nutritional information. Unfortunately there were a lot of missing values so I
# decided to go a different route

food_list = pd.read_csv('food-101/meta/labels.txt', header=None)

In [8]:
food_list.columns = ['food']
food_list['food'] = food_list['food'].str.lower()


In [9]:
food_list.head()

,food
0,apple pie
1,baby back ribs
2,baklava
3,beef carpaccio
4,beef tartare


In [11]:
connection_args = {
    'host': 'localhost',  # We are connecting to our _local_ version of psql
    'dbname': 'pyramid',    # DB that we are connecting to
    'port': 5432          # port we opened on AWS
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)

In [12]:


query = "SELECT * FROM nutrition LIMIT 10;"

pd_sql.read_sql(query, connection)



,index,food_code,display_name,portion_default,portion_amount,portion_display_name,factor,increment,multiplier,grains,...,milk,meats,soy,drybeans_peas,oils,solid_fats,added_sugars,alcohol,calories,saturated_fats
0,0,12350000,sour cream dip,1.00000,.25000,cup,.25000,.25000,1.00000,.04799,...,.00000,.00000,.00000,.00000,.00000,105.64850,1.57001,.00000,133.65000,7.36898
1,1,13110100,"ice cream, regular",1.00000,1.00000,cup,1.00000,.25000,.25000,.00000,...,.29393,.00000,.00000,.00000,.00000,130.99968,95.20488,.00000,267.33000,9.03070
2,2,13110120,"ice cream, rich",1.00000,1.00000,cup,1.00000,.25000,.25000,.00000,...,.25160,.09620,.00000,.00000,.00000,213.06672,96.10340,.00000,368.52000,15.28840
3,3,13110200,soft serve ice cream,1.00000,1.00000,cup,1.00000,.25000,.25000,.00000,...,.38233,.00000,.00000,.00000,.00000,170.39808,123.83793,.00000,347.73000,11.74670
4,4,13120100,"ice cream bar, chocolate covered",1.00000,1.00000,bar (bar = 3 fl oz),1.00000,.50000,.50000,.00000,...,.09744,.00000,.00000,.00000,.00000,112.32144,45.54761,.00000,169.12000,9.58720
5,5,13120100,"ice cream bar, chocolate covered",2.00000,1.00000,klondike krispy bar,1.00000,.50000,.50000,.00000,...,.16008,.00000,.00000,.00000,.00000,184.52808,74.82821,.00000,277.84000,15.75040
6,6,13120500,ice cream sandwich,1.00000,1.00000,sandwich,1.00000,.50000,.50000,.53454,...,.09912,.00000,.00000,.00000,.00000,49.46265,48.40002,.00000,143.37000,3.23320
7,7,13130300,light ice cream,1.00000,1.00000,cup,1.00000,.25000,.25000,.00000,...,.75980,.00000,.00000,.00000,.00000,45.60372,70.88353,.00000,216.15000,3.39290
8,8,13130600,light soft serve ice cream,1.00000,1.00000,cup,1.00000,.25000,.25000,.00000,...,.71750,.00000,.00000,.00000,.00000,39.65850,.00000,.00000,220.50000,2.85250
9,9,13135000,light ice cream sandwich,1.00000,1.00000,sandwich,1.00000,.50000,.50000,.54360,...,.26520,.00000,.00000,.00000,.00000,21.21120,41.22339,.00000,129.60000,1.36800


In [16]:
for ii in range(0, len(food_list)):

    query= "SELECT * FROM nutrition WHERE display_name LIKE '%" + food_list.at[ii, 'food'] + "%';"


    print(pd_sql.read_sql(query, connection))


   index food_code                  display_name portion_default  \
0   1832  53301000          apple pie, two crust         1.00000   
1   1833  53301500  apple pie with crumb topping         1.00000   

  portion_amount   portion_display_name   factor increment multiplier  \
0        1.00000  piece (1/8 of 9" pie)  1.00000    .50000     .50000   
1        1.00000  piece (1/8 of 9" pie)  1.00000    .50000     .50000   

    grains      ...          milk   meats     soy drybeans_peas     oils  \
0  2.00400      ...        .00000  .00000  .00000        .00000  3.41700   
1  1.68300      ...        .00000  .00000  .00000        .00000   .00000   

  solid_fats added_sugars alcohol   calories saturated_fats  
0    5.48100     62.22031  .00000  355.50000        5.70000  
1  121.90500    112.95717  .00000  363.00000        2.74500  

[2 rows x 27 columns]
Empty DataFrame
Columns: [index, food_code, display_name, portion_default, portion_amount, portion_display_name, factor, increment, multi

[0 rows x 27 columns]
Empty DataFrame
Columns: [index, food_code, display_name, portion_default, portion_amount, portion_display_name, factor, increment, multiplier, grains, whole_grains, vegetables, orange_vegetables, drkgreen_vegetables, starchy_vegetables, other_vegetables, fruits, milk, meats, soy, drybeans_peas, oils, solid_fats, added_sugars, alcohol, calories, saturated_fats]
Index: []

[0 rows x 27 columns]
Empty DataFrame
Columns: [index, food_code, display_name, portion_default, portion_amount, portion_display_name, factor, increment, multiplier, grains, whole_grains, vegetables, orange_vegetables, drkgreen_vegetables, starchy_vegetables, other_vegetables, fruits, milk, meats, soy, drybeans_peas, oils, solid_fats, added_sugars, alcohol, calories, saturated_fats]
Index: []

[0 rows x 27 columns]
   index food_code           display_name portion_default portion_amount  \
0   1439  27246100  chicken and dumplings         1.00000         .50000   

  portion_display_name  factor 

   index food_code                              display_name portion_default  \
0    855  63409010                                 guacamole         1.00000   
1   1015  58105000  chicken fajitas with veggies & guacamole         1.00000   
2   1016  58105000  chicken fajitas with veggies & guacamole         2.00000   

  portion_amount portion_display_name   factor increment multiplier   grains  \
0         .25000                  cup   .25000    .25000    1.00000   .00000   
1        1.00000               fajita  1.00000    .50000     .50000  2.10512   
2        1.00000    mcdonald's fajita  1.00000    .50000     .50000   .77408   

       ...          milk    meats     soy drybeans_peas    oils solid_fats  \
0      ...        .00000   .00000  .00000        .00000  .00000     .00000   
1      ...        .00000  1.50748  .00000        .00000  .00396   36.18621   
2      ...        .00000   .55432  .00000        .00000  .00146   13.30614   

  added_sugars alcohol   calories saturated_f

   index food_code                  display_name portion_default  \
0   1078  58130010                  meat lasagna         1.00000   
1   1079  58130010                  meat lasagna         2.00000   
2   1080  58130320  spinach lasagna (vegetarian)         1.00000   
3   1081  58130320  spinach lasagna (vegetarian)         2.00000   

  portion_amount       portion_display_name   factor increment multiplier  \
0        1.00000  piece (about 2-1/2" x 4")  1.00000    .50000     .50000   
1        1.00000                        cup  1.00000    .50000     .50000   
2        1.00000  piece (about 2-1/2" x 4")  1.00000    .50000     .50000   
3        1.00000                        cup  1.00000    .50000     .50000   

    grains      ...          milk   meats     soy drybeans_peas    oils  \
0  1.25660      ...        .70452  .69010  .00000        .00000  .00000   
1  1.52500      ...        .85500  .83750  .00000        .00000  .00000   
2  1.30752      ...        .73321  .00000  .0000

    index food_code                      display_name portion_default  \
0    1018  58106220          cheese pizza, thin crust         1.00000   
1    1019  58106220          cheese pizza, thin crust         2.00000   
2    1020  58106220          cheese pizza, thin crust         2.00000   
3    1021  58106220          cheese pizza, thin crust         2.00000   
4    1022  58106220          cheese pizza, thin crust         2.00000   
5    1023  58106230         cheese pizza, thick crust         1.00000   
6    1024  58106230         cheese pizza, thick crust         2.00000   
7    1025  58106230         cheese pizza, thick crust         2.00000   
8    1026  58106230         cheese pizza, thick crust         2.00000   
9    1027  58106230         cheese pizza, thick crust         2.00000   
10   1028  58106230         cheese pizza, thick crust         2.00000   
11   1029  58106230         cheese pizza, thick crust         2.00000   
12   1030  58106320       vegetable pizza, thin cru

   index food_code                         display_name portion_default  \
0   1372  26317120            baked or broiled scallops         1.00000   
1   1373  26317120            baked or broiled scallops         2.00000   
2   1374  26317120            baked or broiled scallops         2.00000   
3   1375  26317140  fried scallops (floured or breaded)         1.00000   
4   1376  26317140  fried scallops (floured or breaded)         2.00000   
5   1377  26317140  fried scallops (floured or breaded)         2.00000   

  portion_amount  portion_display_name   factor increment multiplier  grains  \
0         .50000           cup, cooked   .50000    .25000     .50000  .00000   
1        1.00000               scallop  1.00000   1.00000    1.00000  .00000   
2        4.00000  ounces after cooking  4.00000   1.00000     .25000  .00000   
3         .50000           cup, cooked   .50000    .25000     .50000  .40870   
4        1.00000      scallop (0.6 oz)  1.00000   1.00000    1.00000  .097